### 7.1 编写可接受任意数量参数的函数

In [1]:
# 要编写一个可接收任意数量的位置参数的函数，可使用以*开头的参数。
def avg(first, *rest):
    return (first + sum(rest)) / (1 + len(rest))

print(avg(1,2))
print(avg(1,2,3,4))

1.5
2.5


In [2]:
# 要要编写一个可接收任意数量的关键字参数的函数，可使用以**开头的参数。
import html
def make_element(name, value, **attrs):
    keyvals = [' %s="%s"' % item for item in attrs.items()]
    attr_str = ''.join(keyvals)
    element = '<{name}{attrs}>{value}</{name}>'.format(
    name=name,
    attrs=attr_str,
    value=html.escape(value))
    return element
e1 = make_element('item', 'Albatross', size='large', quantity=6)
print(e1)
e2 = make_element('p', '<spam>')
print(e2)

<item size="large" quantity="6">Albatross</item>
<p>&lt;spam&gt;</p>


在函数定义中，以*打头的参数只能作为最后一个位置参数出现， 

而以\*\*打头的参数只能作为最后一个参数出现。  
微妙的是在*打头的参数后仍可以有其他参数出现，*args之后的参数只能作为关键字参数使用。

### 7.2 编写只接受关键字参数的函数

将关键字参数放置在以\*打头的参数或者是一个单独的\*之后

In [3]:
def recv(maxsize, *, block):
    'Receive a message'
    pass

# recv(1024, True)  #TypeError
recv(1024, block=True)   #ok

In [4]:
def minimum(*values, clip=None):
    m = min(values)
    if clip is not None:
        m = clip if clip > m else m
    return m

print(minimum(1,5,2,-5,10))
print(minimum(1,5,2,-5,10,clip=0))

-5
0


### 7.3 将元数据函数信息附加到函数参数上

我们可以为函数参数符加一些额外的信息，这样他人可以对函数的使用方法有更多的认识和了解。  
Python解释器并不会符加任何语法意义到这些参数注解上。

In [5]:
def add(x:int, y:int) -> int:
    return x + y

print(help(add))
print(add(2,4))

Help on function add in module __main__:

add(x:int, y:int) -> int

None
6


尽管可以将任何类型的对象作为函数注解符加到函数定义上（数字、字符串、实例等），  
但是通常只有类和字符串才显得最优

### 7.4 从函数中返回多个值

In [6]:
# 返回元组即可
def myfun():
    return 1,2,3

### 7.5 定义带有默认参数的函数

表面看定义一个带有可选参数的函数是非常简单，只需要在定义中为参数赋值，并确保默认参数出现在最后即可。如下：

In [7]:
def spam(a, b=42):
    print(a,b)

spam(1)
spam(1,2)

1 42
1 2


如果默认值是可变容器的话，如列表、集合或者字典，那么应该把None作为默认值，代码应该像这样写：

In [8]:
def spam(a, b=None):
    if b is None:
        b = []
    print(a, b)
    
spam(1)
spam(1, 2)

1 []
1 2


如果不打算提供一个默认值，只想编写代码来检测可选参数是否被赋予了某个特定值，那么可以采用下面这种方法：

In [10]:
_no_value = object()
def spam(a, b=_no_value):
    if b is _no_value:
        print("No b value supplied")
    else:
        print(a, b)
    
spam(1)
spam(1, 2)
spam(1, None)

No b value supplied
1 2
1 None


### 7.6 定义匿名或内联函数

我们需要提供一个短小的回调函数为sort()这样的操作所用，又不想通过def语句编写单行函数。  
这种简单函数可以通过lambda表达式来替代

In [12]:
add = lambda x, y: x + y
add(2,3)

5

一般lambda表达式用在如排序或对数据进行整理时：

In [14]:
names = ['David Beazley', 'Brian Jones', 'Raymond Hettinger', 'Ned Batchelder']
# 英文姓名按姓氏排序
sorted(names, key=lambda name: name.split()[-1].lower())

['Ned Batchelder', 'David Beazley', 'Raymond Hettinger', 'Brian Jones']

### 7.7 在匿名函数中绑定变量的值

如果希望匿名函数可以在定义的时候绑定变量，并保持值不变，可以将那个值作为默认参数实现。

In [15]:
x = 10
a = lambda y, x=x: x+y
x = 20
b = lambda y, x=x: x+y
print(a(10))
print(b(10))

20
30


通过列表推导式来创建一列lambda表达式，或者在一个循环中期望lambda表达式能够在定义时候记住迭代变量，如下：

In [16]:
funcs = [lambda x, n=n: x+n for n in range(5)]
for f in funcs:
    print(f(0))

0
1
2
3
4


### 7.8 让带有N个参数的可调用对象以较少的参数形式调用

用functools.partial()

In [17]:
def spam(a,b,c,d):
    print(a, b, c, d)
    
from functools import partial
s1 = partial(spam, 1)
s1(2,3,4)
s1(4,5,6)

s2 = partial(spam, d=42)
s2(1,2,3)

1 2 3 4
1 4 5 6
1 2 3 42


这种技术可以将看似不兼容的代码结合起来。

假设有一列以元组(x,y)来表示的点坐标。可以用下面的函数来计算两点之间的距离。

In [2]:
points = [(1,2), (3,4), (5,6), (7,8)]
import math
def distance(p1, p2):
    x1,y1 = p1
    x2,y2 = p2
    return math.hypot(x2-x1, y2-y1)

现在假设想根据这些点之间的距离来对他们排序。列表的sort()方法可接受一个key参数，它可以用来做自定义的排序处理。但是它只能和接受单参数的函数一起工作（因此和distance（）是不兼容的）。我们用partial()来解决这个问题：

In [5]:
pt = (4, 3)
from functools import partial
points.sort(key=partial(distance,pt))
print(points)

[(3, 4), (1, 2), (5, 6), (7, 8)]


partial()常常可用来调整其他库中用到的回调函数的参数签名。  
这里有一段代码利用multiprocessing模块以异步方式计算某个结果，并将这个结果传递给一个回调函数。该回调函数可接受这个结果以及一个可选的日志参数。  

```python
def output_result(result, log=None):
    if log is not None:
        log.debug('Got: %r', result)
        
# A simple function
def add(x, y):
    return x + y

import logging
from multiprocessing import Pool
from functools import partial
if __name__ = "__main__":
    logging.basicConfig(level=logging.DEBUG)
    log = logging.getLogger('test')

    p = Pool()
    p.apply_async(add, (3, 4), callback=partial(output_result, log=log))
    p.close()
    p.join()
```

编写网络服务器程序

In [ ]:
from socketserver import StreamRequestHandler, TCPServer

class EchoHandler(StreamRequestHandler):
    def handle(self):
        for line in self.rfile:
            self.wfile.write(b'GOT:' + line)
            
serv = TCPServer(('', 15000), EchoHandler)
serv.serv_forever()

假如我们想在EchoHandler类中增加一个__init__()方法， 让它接受一个额外的配置参数。如下：

In [ ]:
class EchoHandler(StreamRequestHandler):
    # ack is added keyword-only argument. *args, **kwargs are
    # any normal parameters supplied (which are passed on)
    def __init__(self, *args, ack, **kwargs):
        self.ack = ack
        super().__init__(*args, **kwargs)
    def handle(self):
        for line in self.rfile:
            self.wfile.write(self.ack + line)

事实上，改动之后没法简单地将其插入到TCPServer类中了。但是，可以利用partial()能轻松解决这个问题。

In [ ]:
from functools import partial
serv = TcpServer(('', 15000), partial(EchoHandler, ack=b'RECEIVED:'))
serv.serve_forever()

有时候也可以通过lambda表达式来替代partial().

```python
points.sort(key=lambda p: distance(pt, p))
p.apply_async(add, (3, 4), callback=lambda result: output_sesult(result, log))
serv = TCPServer(('', 15000), 
                lambda *args, **kwargs: EchoHandler(*args, ack=b'RECEIVED:', **kwargs))
```